<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/104_%E5%BC%8F%E3%81%A8%E6%96%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

式と文
======

Python 識別子
-------------

Python で**識別子**（identifier）は、変数、関数、クラスなどの名前（name）のことである。何が識別子とされるかは [BNF](https://ja.wikipedia.org/wiki/バッカス・ナウア記法) という特殊な形式で正確に記述されている（[公式ドキュメント](https://docs.python.org/ja/3/reference/lexical_analysis.html#identifiers)）。ASCII 範囲（U+0001..U+007F）の中では、以下のようなルールになる：

  1. 識別子は大文字・小文字の A〜Z またはアンダースコア `'_'` で始まることができる。
  2. その後に、0 個以上の大文字・小文字、アンダースコア、または数字 0〜9 が続く。

これは他の多くの言語（C、Java、JavaScript など）と共通の規則であり、プログラマーが直感的に理解しやすいものとなっている。注意点は以下の通り。

  * アンダースコア以外の記号は、識別子に使用できない。例えば、ハイフン `'-'`、空白 `' '`、ドルマーク `'$'`、アットマーク `'@'` はすべて使えない。
  * 識別子は大文字・小文字を区別する（case-sensitive）。
  * 識別子は数字から始められない。

識別子の先頭文字に数字が使えない理由は、ルールを単純化するためである。もしルール 1 で数字が使えるとすると、ルール 2 により `1` や `12` なども識別子となり、これでは数値なのか識別子なのか判別できなくなってしまうので、ルールを付け加える必要がある。

識別子のルールは上記の 2 つだけなので、1 文字だけの識別子が許容され、また長さに上限はない。

実は、ASCII の範囲外にある Unicode 文字（漢字や仮名、全角英数字など）も識別子に使える。しかし、ASCII の範囲外では、異なるコードポイントを同じ文字として扱う「Unicode 正規化」と呼ばれる処理が行われた後で解釈されるので、見た目が違う識別子が同じ識別子として扱われることがあり、予期せぬ変数の衝突や上書きが起こる。

In [ ]:
x = 1
ｘ = 2   # 全角の x (U+FF58)
print(f"{x=}")  # x の値が上書きされる

x=2


もとより、オープンソースなプロジェクトで ASCII 以外の文字の使用を控えるのは当然であり、PEP 8 でも識別子には ASCII のみを使わなければならないとしている。

一部の識別子は、Python 言語における**キーワード**（keyword）として予約され、通常の識別子として使うことはできないとされる。

Python のキーワードは、`keyword.kwlist` で確認できる。

In [ ]:
import keyword
for i, k in enumerate(keyword.kwlist):
    print(f"{k:10}", end="")
    if (i + 1) % 12 == 0:
        print("\n")

False     None      True      and       as        assert    async     await     break     class     continue  def       

del       elif      else      except    finally   for       from      global    if        import    in        is        

lambda    nonlocal  not       or        pass      raise     return    try       while     with      yield     

Python 式
---------

Python の構文で、1 つの処理を書くものとして完結しているものを**文**（statement）という。**文**のうち、評価された値を持つものを**式**（expression）という。つまり、Python では式と式でない文があって、たいていの構文が値を返す式であるという関数型言語の状況とは大きく異なる。

### 帰納的定義 ###

Python で何が式とされるかは BNF で正確に記述されている（[公式ドキュメント](https://docs.python.org/ja/3/reference/expressions.html)）。ここでは、BNF による記述を自然言語に翻訳して、Python 式の帰納的定義を試みる。これは「〇〇は式である」という表現を重ねて、「以上が式の全てである」と締める形の定義である。

  1. 次はアトム（atom）と呼ばれ、式である
      * 名前──値は名前に束縛されたオブジェクト（変数、関数、クラス、モジュールなど）
      * リテラル（文字列、バイト列、整数、浮動小数点数、虚数など）──値はリテラル値
      * 丸括弧 `()` で囲ったもの──少なくとも一つのカンマが入っていた場合の値はタプル
      * リスト・集合・辞書の表示（式のリストまたは内包表記による）──値は各オブジェクト
      * ジェネレーター式──値はジェネレーターオブジェクト
      * yield 式と yield from 式（`(yield <expr>)` や `(yield from <expr>)` の形式）──値は後ろに置かれた式 expr の評価結果
  2. 次はプライマリ（primary）と呼ばれ、式である
      * アトム
      * 属性参照（プライマリと名前を `.` でつなげたもの）
      * 添字表記（プライマリの後ろに `[整数の値をもつ式]` を書いたもの）
      * スライス表記（プライマリの後ろに `[スライスオブジェクトの値をもつ式]` を書いたもの）
      * 呼び出し（プライマリの後ろに `(引数のリスト)` を書く形で、関数・メソッドなどを呼び出すもの）
  3. await 式（`await プライマリ`）は式である
  4. プライマリと await 式は、単独でも演算式と呼ばれる。演算式に対する算術演算とビット演算はすべて演算式と呼ばれ、式である（値は演算結果）
      * べき乗演算： `**`  （※注意： Python の演算子でこれだけが右結合的である）
      * 単項算術演算： `-`（符号反転）、`+`（値そのまま）
      * 単項ビット演算： `~`（ビット反転）
      * 二項算術演算： `*`、`@`、`/`、`//`、`%`、`+`、`-`
      * シフト演算： `<<`、`>>`
      * 二項ビット演算： `&`（AND）、`^`（XOR）、`|`（OR）
  5. 演算式は、単独でも比較式と呼ばれる。比較式の比較はすべて比較式と呼ばれ、式である（値は `True` または `False`）
      * 値の比較： `<`、`>`、`==`、`>=` `<=`、`!=`
      * 所属関係： `in`、`not in`
      * 同一性： `is`、 および `is not`
  6. 比較式は、単独でもブール式と呼ばれる。ブール式に対するブール演算はすべてブール式と呼ばれ、式である
      * `not`──ブール式が真なら `False`、偽なら `True` を返す
      * `and`──左から最初に偽であると判定したブール式（すべて真ならば最後に評価したブール式）を評価した結果値を返す
      * `or`──左から最初に真であると判定したブール式（すべて偽ならば最後に評価したブール式）を評価した結果値を返す
  7. ブール式は、単独でも条件式と呼ばれる。ブール式 `A`、ブール式 `B`、式 `C` に対する条件演算 `A if B else C` も条件式と呼ばれ、式である（値は `B` が真なら `A` の評価結果、`B` が偽なら `C` の評価結果）
  8. ラムダ式（`lambda [引数リスト] : 式`）は式である──値は `式` の評価結果を返す関数オブジェクト

以上が式の全てである。

また、**代入式**（assignment expression）と**式のリスト**を次のように定義する。

  * 式は、単独でも代入式と呼ばれる。任意の名前 `name` と式 `exp` に対する `name := exp` の形（`:=` は**セイウチ演算子**と呼ばれる）はすべて代入式と呼ばれ、右辺 `exp` を評価した結果値を返す。式は左から右へと順に評価されるのに対して、`:=` 演算子については右辺が左辺よりも先に評価されることに注意する。
  * 式は、単独でも式のリストと呼ばれる。カンマ区切りで式を並べたもの、および、式（のリスト）の末尾にカンマを加えたものはすべて式のリストと呼ばれる。リスト表示や辞書表示の一部になっているものを除き、少なくとも 1 つのカンマを含む式のリストはタプルを返す。タプルの長さは、リストにある式の数に等しくなる。式は左から右へ評価される。式の前に付いたアスタリスク `*` はイテラブル（文字列、タプル、リストなど）のアンパックを意味し、各要素が新しいタプルに入れ込まれる。

「代入式」と「式のリスト」が他のプログラミング言語における「式」に相当する。Python 式の定義が「代入式」と「式のリスト」の二本立てになっているのは、`:=` 演算子が Python 3.8 になって導入された際に既存の Python コードに影響が無いよう、もともと「式のリスト」まで構成された式の構文規則に代入式の構文を後付けする形にしたからだと思われる。BNF は次のようになっている:

``` text
assignment_expression ::=  [identifier ":="] expression
expression_list       ::=  expression ("," expression)* [","]
expression            ::=  conditional_expression | lambda_expr
```

なお、帰納的定義において演算子が現れた順番は、演算子の優先順位とは異なる。演算子の優先順位については[公式ドキュメント](https://docs.python.org/ja/3/reference/expressions.html#operator-precedence)を参照。

### eval() ###

組み込み関数 `eval()` は、渡された文字列を Python 式として構文解析して評価した結果を返す。渡された文字列が式でないなら、例外を送出する。

In [ ]:
eval("'a' if True else 'b'")

'a'

Python 文
---------

### 単純文と複合文 ###

Python の文は、**単純文**（simple statement）と**複合文**（compound statement）から構成される。

単純文は、単一の論理行内に収められる文である。次のようなものがある。

  * 式文 （※注意： 「代入式」と「式のリスト」は文である）
  * 代入文
  * assert 文
  * pass 文
  * del 文
  * return 文
  * yield 文 （※注意: yield 式文の丸括弧を省略した形）
  * raise 文
  * break 文
  * continue 文
  * import 文
  * global 文
  * nonlocal 文
  * type 文 （※注意: Python 3.12 で追加）

複合文は、複数行にまたがって書かれる文である。次のようなものがある。

  * if 文（オプションとして elif 節・else 節）
  * while 文（オプションとして else 節）
  * for 文（オプションとして else 節）
  * try 文（オプションとして except 節・else 節・finally 節） （※注意: except 節と finally 節はどちらか一方が必ず必要）
  * with 文 （オプションとして `as 変数`）
  * match-case 文 （※注意： Python 3.10 で追加。`match` と `case` は match-case 文以外の場面では名前として使用できる）
  * 関数定義 `def`
  * クラス定義 `class`
  * コルーチン `async`

### exec() ###

組み込み関数 `exec()` は、渡された文字列を Python 文として構文解析して、構文エラーにならない限り実行する。戻り値は `None`。

In [ ]:
exec("a = 2; b = 3; print(a + b)")

5


### assert 文 ###

assert 文は、プログラムの動作中に「ある条件が成り立つこと」を確認（検証）するための文である。条件が成り立たなければ `AssertionError` 例外を発生させて、プログラムの異常な状態を検出することができる。

assert 文の構文は次のとおり:

``` python
assert 式[, 式]
```

文 `assert 式` は、`式` が真であれば何も起こらず、`式` が偽であれば `AssertionError` 例外を発生させる。2 つ目の `式` が与えられた場合、それはエラーメッセージに使用される。

`assert` がなぜ `exec()` のような関数ではなく文なのかというと、最適化のためである。実は、assert 文は Python バイトコード生成時に `__debug__` という組み込み定数を使って特別に処理される。

文 `assert 式` から生成される Python バイトコードを Python で疑似的に表現すれば次のように書ける:

``` python
if __debug__:
    if not 式: raise AssertionError
```

文 `assert 式, 式2` から生成される Python バイトコードを Python で疑似的に表現すれば次のように書ける:

``` python
if __debug__:
    if not 式: raise AssertionError(式2)
```

`__debug__` は、最適化を意味するインタープリターのフラグ `-O` を指定したとき `False` で、それ以外の場合は `True` である。したがって、最適化フラグ `-O` 付きなら `式`、`式2` は評価されない。これは通常の関数では実装することができない（もし `assert(式,[ 式2])` のような関数実装なら、最初に引数の `式,[ 式2]` が評価されてしまう）。

本番環境で最適化フラグ `-O` を指定すると、 assert 文は全く実行されないからエラーチェックにならない。したがって、assert 文はテスト環境でエラーを検出するために使用されるべきである。